In [ ]:
import openrouteservice
import folium
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import googlemaps

g_api = open("gmaps_api.txt",'r')
g_api_key = g_api.read()

def reverse_geocode(latitude, longitude):
    geolocator = Nominatim(user_agent="reverse_geocode_app")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)

    if location:
        address = location.address
        return address
    else:
        return "Location not found"


def calculate_eta(api_key, start_coords, end_coords):
    client = openrouteservice.Client(key=api_key)
    
    # Define the coordinates of the start and end points
    start = start_coords
    end = end_coords
    
    # Calculate the route and extract ETA
    route = client.directions(coordinates=[start, end], profile='driving-car', format='geojson')
    
    if "features" in route:
        duration_seconds = route['features'][0]['properties']['segments'][0]['duration']
        duration_minutes = duration_seconds / 60.0
        return duration_minutes
    else:
        return None
    

# function to calculate the actual distance by road.
def calculate_actual_distance(g_api_key,start_coords,end_coords) :
    gmaps = googlemaps.Client(key=g_api_key)
    # Get directions between the two locations
    directions_result = gmaps.directions(start_coords[::-1], end_coords[::-1] , mode="driving")

    if directions_result:
        # Extract the distance in meters from the result
        distance_meters = directions_result[0]["legs"][0]["distance"]["value"]

        # Convert meters to kilometers
        distance_kilometers = distance_meters / 1000

        return distance_kilometers

    return None


# Replace 'YOUR_API_KEY' with your OpenRouteService API key
api_key = 'api key'

# Coordinates of the start point
start_coords = (12.894285846271835, 77.61339570241948)[::-1]  # Reverse the coordinates

# Coordinates of the safe houses
safe_house_coords = [
    [12.926091697490882, 77.55034182125144][::-1],
    [12.890661182014599, 77.5980490821104][::-1],
    [13.03067966399335, 77.53851060492106][::-1],
    [12.958225634936344, 77.68621387932865][::-1],
    [12.840594637654393, 77.66939000367039][::-1],
    [13.043812030190258, 77.63927634537357][::-1]
]

# Create an OpenRouteService client
client = openrouteservice.Client(key=api_key)

# Create a list to store distances
distances = []

# Calculate distances and print them
c = 1
for safe_house_coord in safe_house_coords:
    distance = calculate_actual_distance(g_api_key,start_coords,safe_house_coord)
    print("distance of current location from safe house",c,"is",distance,"km")
    c=c+1;
    distances.append(distance)


# Create a map
m = folium.Map(location=start_coords[::-1], tiles="cartodbpositron", zoom_start=14)
folium.Marker(location=start_coords[::-1], icon=folium.Icon(color='red')).add_to(m)

# Add markers for safe house locations
for coord in safe_house_coords:
    folium.Marker(location=coord[::-1]).add_to(m)

# add routes 
profile = 'driving-car'

shortest_distance = min(distances)
si = distances.index(shortest_distance)
end_coord_shortest = safe_house_coords[si]

# Request directions and routes
routes = client.directions(
    coordinates=[start_coords, end_coord_shortest],
    profile=profile,
    format='geojson')

# Extract and print the route details
if 'features' in routes:
    for feature in routes['features']:
        geometry = feature['geometry']['coordinates']
        #print("Route coordinates:", geometry)
else:
    print("No route found")

pts_list = []

for i in geometry :
    pts_list.append(i[::-1])

folium.PolyLine(pts_list,colour = "blue",dash_array = 5, opacity = 0.85,tooltip = str(round(shortest_distance, 2))+"Km").add_to(m)

# statistics : 

total_distance = shortest_distance # in kms.
eta = calculate_eta(api_key,start_coords,end_coord_shortest) #in minutes
area_name = reverse_geocode(end_coord_shortest[1], end_coord_shortest[0])
print("\n*area name: ",area_name)
print("Total distance between start and end location: ",total_distance,"km")
print("Estimated time of arrival in minutes: ",eta)
ndrf = 'http://www.ndrf.gov.in/'
emergency_contacts : "https://www.indiacustomercare.com/all-india-emergency-contact-toll-free-numbers"

f = open("data.txt",'w') 
f.write(str(total_distance)+'\n')
f.write(str(eta)+'\n')
f.write(area_name+'\n')
f.close()

# Display the map
m
#m.save('map.html')
